In [1]:
import datetime as dt
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

In [2]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [3]:
Base = automap_base()
Base.prepare(engine, reflect=True)

In [4]:
Measurement = Base.classes.measurement
Station = Base.classes.station

In [5]:
session = Session(engine)

In [6]:
# Create ab app, being sure to pass__name__
app = Flask(__name__)

@app.route('/')
def welcome():
    return (
    '''
    Welcome to the Climate Analysis API!
    Available Routes:
    /api/v1.0/precipitation
    /api/v1.0/stations
    /api/v1.0/tobs
    /api/v1.0/temp/start/end
    ''')    

In [7]:
# Create the route
@app.route("/api/v1.0/precipitation")
# Create precipitation function
def precipitation():
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    precipitation = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= prev_year).all()
    precip = {date: prcp for date, prcp in precipitation}
    return jsonify(precip) 

In [8]:
@app.route("/api/v1.0/temp/<start>")
@app.route("/api/v1.0/temp/<start>/<end>")
# /api/v1.0/temp/<start>/<end> becomes /api/v1.0/temp/2017-06-01/2017-06-30

def stats(start=None, end=None):
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]

    if not end:
        results = session.query(*sel).\
            filter(Measurement.date >= start).\
            filter(Measurement.date <= end).all()
        temps = list(np.ravel(results))
        return jsonify(temps)

    results = session.query(*sel).\
        filter(Measurement.date >= start).\
        filter(Measurement.date <= end).all()
    temps = list(np.ravel(results))
    return jsonify(temps=temps)

In [ ]:
#export FLASK_APP=app.py
#flask run

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/May/2021 23:56:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2021 23:56:38] "GET /api/v1.0/precipitation HTTP/1.1" 200 -
[2021-05-06 23:57:25,047] ERROR in app: Exception on /api/v1.0/temp/2016-10-25 [GET]
Traceback (most recent call last):
  File "/Users/Irei/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/Irei/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/Irei/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/Irei/opt/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/Irei/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
  

127.0.0.1 - - [06/May/2021 23:57:45] "GET /api/v1.0/temp/10-25-2016 HTTP/1.1" 500 -
[2021-05-06 23:58:08,788] ERROR in app: Exception on /api/v1.0/temp/<start> [GET]
Traceback (most recent call last):
  File "/Users/Irei/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/Irei/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/Irei/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/Irei/opt/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/Irei/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/Irei/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1936, in

127.0.0.1 - - [07/May/2021 00:00:08] "GET /api/v1.0/temp/start HTTP/1.1" 500 -
127.0.0.1 - - [07/May/2021 00:00:22] "GET /api/v1.0/temp/start/end HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2021 00:01:58] "GET /api/v1.0/temp//api/v1.0/temp/2017-06-01/2017-06-30 HTTP/1.1" 404 -
127.0.0.1 - - [07/May/2021 00:02:10] "GET /api/v1.0/temp/2017-06-01/2017-06-30 HTTP/1.1" 200 -
